## Preprocessing

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(columns=['EIN', 'NAME'])

In [3]:
# Reviewing unique value counts for binning
application_type_counts = application_df['APPLICATION_TYPE'].value_counts()
classification_counts = application_df['CLASSIFICATION'].value_counts()

In [4]:
# Binning application type and classification
application_types_to_replace = list(application_type_counts[application_type_counts < 500].index)
classification_to_replace = list(classification_counts[classification_counts < 1500].index)

In [5]:
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app, 'Other')

In [6]:
for cls in classification_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls, 'Other')

In [7]:
# Creating dummy variables for categorical data
application_df = pd.get_dummies(application_df)

In [8]:
# Splitting into features and target
X = application_df.drop(columns=['IS_SUCCESSFUL'])
y = application_df['IS_SUCCESSFUL']

In [9]:
# Splitting into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78, stratify=y)


In [10]:
# Scaling the data
scaler = StandardScaler()
X_scaler = scaler.fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [11]:
# Creating the neural network model
nn = tf.keras.models.Sequential()

In [12]:
# Adding layers - experimenting with more neurons and hidden layers
nn.add(tf.keras.layers.Dense(units=100, input_dim=len(X_train.columns), activation='relu'))
nn.add(tf.keras.layers.Dense(units=50, activation='relu'))
nn.add(tf.keras.layers.Dense(units=30, activation='relu'))
nn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

In [13]:
# Compiling the model
nn.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [14]:
# Training the model with increased epochs
model = nn.fit(X_train_scaled, y_train, epochs=150)

Epoch 1/150
804/804 [==============================] - 14s 8ms/step - loss: 0.5673 - accuracy: 0.7220
Epoch 2/150
804/804 [==============================] - 3s 3ms/step - loss: 0.5557 - accuracy: 0.7278
Epoch 3/150
804/804 [==============================] - 3s 3ms/step - loss: 0.5527 - accuracy: 0.7301
Epoch 4/150
804/804 [==============================] - 2s 3ms/step - loss: 0.5514 - accuracy: 0.7310
Epoch 5/150
804/804 [==============================] - 2s 2ms/step - loss: 0.5499 - accuracy: 0.7308
Epoch 6/150
804/804 [==============================] - 2s 2ms/step - loss: 0.5487 - accuracy: 0.7316
Epoch 7/150
804/804 [==============================] - 2s 2ms/step - loss: 0.5482 - accuracy: 0.7324
Epoch 8/150
804/804 [==============================] - 2s 2ms/step - loss: 0.5477 - accuracy: 0.7325
Epoch 9/150
804/804 [==============================] - 2s 2ms/step - loss: 0.5467 - accuracy: 0.7316
Epoch 10/150
804/804 [==============================] - 3s 3ms/step - loss: 0.5464 - accur

In [15]:
# Evaluating the model
model_loss, model_accuracy = nn.evaluate(X_test_scaled, y_test, verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.5801 - accuracy: 0.7311 - 534ms/epoch - 2ms/step
Loss: 0.5800945162773132, Accuracy: 0.7310787439346313
